In [ ]:
import pandas as pd
from google.colab import drive
#SACAR protocols_features_df, NO CAMBIA MUCHO
drive.mount('/content/drive')

train_data = pd.read_parquet("/content/drive/MyDrive/TP2 - ORGANIZACIÓN DE DATOS/dataset_v2/train.parq", engine="pyarrow")

#New features
#https://colab.research.google.com/drive/1M9Hh3YWE5jiA59oerCf6fChVYaFJq3Gp?usp=sharing
one_hot_attack_types_df = pd.read_csv("/content/drive/MyDrive/TP2 - ORGANIZACIÓN DE DATOS/DFs Base Line/one_hot_attack_types_df.csv", engine="pyarrow")  #TP2-Features-One-hot attack_type.ipynb
one_hot_attack_service_df = pd.read_csv("/content/drive/MyDrive/TP2 - ORGANIZACIÓN DE DATOS/DFs Base Line/one_hot_attack_service_df.csv", engine="pyarrow") #TP2-Features-One-hot attack_type.ipynb


##https://colab.research.google.com/drive/16Chf3GEfKOTpDEWehhCE74wFfAcuxWW3?usp=sharing
one_hot_days_df = pd.read_csv("/content/drive/MyDrive/TP2 - ORGANIZACIÓN DE DATOS/DFs Base Line/one_hot_days_df.csv", engine="pyarrow") #TP2-Features-One-hot attack_time(days).ipynb

#https://colab.research.google.com/drive/11Kn4qbTRQ2PmKyYBaApSdACwB2hoCogO?usp=sharing
ports_features_df = pd.read_csv("/content/drive/MyDrive/TP2 - ORGANIZACIÓN DE DATOS/New Features/ports_features_df.csv", engine="pyarrow") #TP2-Features-One-hot open_ports.ipynb

#https://colab.research.google.com/drive/1EKOTO-a7iGCuDPDbBa-casQY9oJCXeg0?usp=sharing
protocols_features_df = pd.read_csv("/content/drive/MyDrive/TP2 - ORGANIZACIÓN DE DATOS/New Features/protocols_features_df.csv", engine="pyarrow") #TP2-Features-protocol_count.ipynb


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_data.head()

,attack_time,watcher_country,watcher_as_num,watcher_as_name,attacker_country,attacker_as_num,attacker_as_name,attack_type,watcher_uuid_enum,attacker_ip_enum,label
0,2023-07-31 07:17:51+00:00,DE,34011.0,Host Europe GmbH,TR,47721.0,Murat Aktas,http:exploit,0,6466,0
1,2023-07-31 07:17:51+00:00,DE,34011.0,Host Europe GmbH,TR,47721.0,Murat Aktas,http:spam,0,6466,0
2,2023-07-31 07:17:49+00:00,DE,20886.0,bn:t Blatzheim Networks Telecom GmbH,DE,51167.0,Contabo GmbH,http:bruteforce,2,4637,0
3,2023-07-31 07:17:49+00:00,DE,20886.0,bn:t Blatzheim Networks Telecom GmbH,DE,51167.0,Contabo GmbH,http:spam,2,4637,0
4,2023-07-31 07:17:49+00:00,DE,20886.0,bn:t Blatzheim Networks Telecom GmbH,DE,51167.0,Contabo GmbH,http:exploit,2,4637,0


## Preparación del dataset

In [ ]:
label_df = train_data.drop_duplicates(["attacker_ip_enum", "label"]).set_index("attacker_ip_enum").loc[:, ['watcher_as_num','attacker_as_num','attacker_country','watcher_country',"label"]]

#Conviero los '...as_num' a categorica para luego poder aplicarle un encoding acorde
label_df['watcher_as_num'] = label_df['watcher_as_num'].astype('category')
label_df['attacker_as_num'] = label_df['attacker_as_num'].astype('category')


dataset = pd.concat(
    [
        one_hot_attack_service_df,
        one_hot_attack_types_df,
        one_hot_days_df,
        ports_features_df,
        protocols_features_df,
        label_df
    ],
    axis=1,
    join="inner",
)

if 'attacker_ip_enum' in dataset.columns:
    dataset = dataset.drop('attacker_ip_enum', axis=1)


In [ ]:
dataset.head()

,database,ftp,http,pop3/imap,sip,smb,ssh,tcp,telnet,unknown_service,...,port_8080,open_ports_count,protocol_tcp,protocol_udp,protocols_count,watcher_as_num,attacker_as_num,attacker_country,watcher_country,label
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,7162.0,14618.0,US,BR,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,203417.0,27699.0,BR,PL,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,208857.0,3320.0,DE,AT,0
3,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,20473.0,21826.0,VE,SG,0
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,24940.0,25019.0,SA,DE,0


In [ ]:
dataset['label'].value_counts()

0    107293
1      2146
Name: label, dtype: int64

In [ ]:
dataset.columns

Index(['database', 'ftp', 'http', 'pop3/imap', 'sip', 'smb', 'ssh', 'tcp',
       'telnet', 'unknown_service', 'windows', 'bruteforce', 'crawl',
       'exploit', 'scan', 'spam', 'unknown_type', 'Friday', 'Monday',
       'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday', 'port_1080',
       'port_110', 'port_1194', 'port_143', 'port_1701', 'port_1723',
       'port_20', 'port_21', 'port_22', 'port_23', 'port_25', 'port_500',
       'port_53', 'port_8080', 'open_ports_count', 'protocol_tcp',
       'protocol_udp', 'protocols_count', 'watcher_as_num', 'attacker_as_num',
       'attacker_country', 'watcher_country', 'label'],
      dtype='object')

## Train y test

In [ ]:
!pip install --upgrade imbalanced-learn scikit-learn
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split

In [ ]:
train, test = train_test_split(dataset, test_size=0.3, random_state=42)

## Mean encoding de 'attacker_as_num' y 'watcher_as_num'

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split


# Calcular las medias por categoría en el conjunto de entrenamiento
means_attacker_as_num = train.groupby('attacker_as_num')['label'].mean()
means_watcher_as_num = train.groupby('watcher_as_num')['label'].mean()

train['attacker_as_num'] = train['attacker_as_num'].map(means_attacker_as_num)
test['attacker_as_num'] = test['attacker_as_num'].map(means_attacker_as_num)

train['watcher_as_num'] = train['watcher_as_num'].map(means_watcher_as_num)
test['watcher_as_num'] = test['watcher_as_num'].map(means_watcher_as_num)

In [ ]:
train.head(10)

,database,ftp,http,pop3/imap,sip,smb,ssh,tcp,telnet,unknown_service,...,port_8080,open_ports_count,protocol_tcp,protocol_udp,protocols_count,watcher_as_num,attacker_as_num,attacker_country,watcher_country,label
45062,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0.000000,0.000638,US,AR,0
136282,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0.028419,0.000000,JP,JP,0
4285,0,0,1,0,0,0,0,0,0,0,...,0,1,1,0,1,0.029615,0.002558,CN,SG,0
3477,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0.004882,0.000000,AT,AT,0
84069,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0.000000,0.000000,US,FR,0
23152,0,0,1,0,0,0,0,0,0,0,...,1,3,1,0,1,0.000000,0.000000,IN,PL,0
112485,0,0,1,0,0,0,0,0,0,0,...,0,1,1,0,1,0.024994,0.000000,BR,US,0
11274,0,0,1,0,0,0,0,0,0,0,...,0,1,1,0,1,0.001248,0.000000,DE,DE,0
60830,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0.023983,0.000000,DE,US,0
67344,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0.018787,0.000000,US,US,0


## Separacion en x e y

In [ ]:
X_train_raw = train.drop(["label"], axis=1)
y_train_raw = train["label"]
X_test_raw = test.drop(["label"], axis=1)
y_test_raw = test["label"]

In [ ]:
X_train_raw

,database,ftp,http,pop3/imap,sip,smb,ssh,tcp,telnet,unknown_service,...,port_53,port_8080,open_ports_count,protocol_tcp,protocol_udp,protocols_count,watcher_as_num,attacker_as_num,attacker_country,watcher_country
45062,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.000000,0.000638,US,AR
136282,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.028419,0.000000,JP,JP
4285,0,0,1,0,0,0,0,0,0,0,...,0,0,1,1,0,1,0.029615,0.002558,CN,SG
3477,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.004882,0.000000,AT,AT
84069,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.000000,0.000000,US,FR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73821,0,0,0,0,0,0,1,0,0,0,...,0,0,1,1,0,1,0.000000,0.007418,CN,KG
103546,0,0,1,0,0,0,0,0,0,0,...,0,0,1,1,0,1,0.035199,0.000000,MX,US
139903,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.035199,0.007418,CN,US
1180,0,0,1,0,0,0,0,0,0,0,...,0,0,6,1,1,2,0.000000,0.000000,US,SE


## Binary encoding de 'attacker_country' y 'watcher_country' e imputacion de valores nulos

In [ ]:
pip install category_encoders

In [ ]:
from category_encoders.binary import BinaryEncoder
from sklearn.impute import SimpleImputer

# Como los valores nan son categóricos, los voy a reemplazar con los más comunes
columns_with_nans = ['attacker_country', 'watcher_country', 'watcher_as_num', 'attacker_as_num']

# Imputar los valores nulos
imputer = SimpleImputer(strategy='most_frequent')
X_train_raw[columns_with_nans] = imputer.fit_transform(X_train_raw[columns_with_nans])
X_test_raw[columns_with_nans] = imputer.transform(X_test_raw[columns_with_nans])

# Crear una nueva columna combinando "attacker_country" y "watcher_country"
X_train_raw['combined_countries'] = X_train_raw['attacker_country'] + '-' + X_train_raw['watcher_country']
X_test_raw['combined_countries'] = X_test_raw['attacker_country'] + '-' + X_test_raw['watcher_country']

# Crear un objeto BinaryEncoder
encoder = BinaryEncoder(cols=['combined_countries'], drop_invariant=True)

# Aplicar el BinaryEncoder a la nueva columna
X_train_enc = encoder.fit_transform(X_train_raw)
X_test_enc = encoder.transform(X_test_raw)

# Eliminar las columnas originales "attacker_country" y "watcher_country"
X_train_enc.drop(['attacker_country', 'watcher_country'], axis=1, inplace=True)
X_test_enc.drop(['attacker_country', 'watcher_country'], axis=1, inplace=True)

In [ ]:
X_test_raw.isnull().sum()


database              0
ftp                   0
http                  0
pop3/imap             0
sip                   0
smb                   0
ssh                   0
tcp                   0
telnet                0
unknown_service       0
windows               0
bruteforce            0
crawl                 0
exploit               0
scan                  0
spam                  0
unknown_type          0
Friday                0
Monday                0
Saturday              0
Sunday                0
Thursday              0
Tuesday               0
Wednesday             0
port_1080             0
port_110              0
port_1194             0
port_143              0
port_1701             0
port_1723             0
port_20               0
port_21               0
port_22               0
port_23               0
port_25               0
port_500              0
port_53               0
port_8080             0
open_ports_count      0
protocol_tcp          0
protocol_udp          0
protocols_count 

In [ ]:
X_train_enc.head()

,database,ftp,http,pop3/imap,sip,smb,ssh,tcp,telnet,unknown_service,...,combined_countries_2,combined_countries_3,combined_countries_4,combined_countries_5,combined_countries_6,combined_countries_7,combined_countries_8,combined_countries_9,combined_countries_10,combined_countries_11
45062,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
136282,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4285,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
3477,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
84069,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1


## Normalizado

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

sc = StandardScaler()
X_train = sc.fit_transform(X_train_enc)
X_test = sc.transform(X_test_enc)

y_train = y_train_raw
y_test  = y_test_raw

In [ ]:
X_train_enc.head()


,database,ftp,http,pop3/imap,sip,smb,ssh,tcp,telnet,unknown_service,...,combined_countries_2,combined_countries_3,combined_countries_4,combined_countries_5,combined_countries_6,combined_countries_7,combined_countries_8,combined_countries_9,combined_countries_10,combined_countries_11
45062,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
136282,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4285,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
3477,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
84069,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1


## Modelo

In [ ]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score

In [ ]:
import xgboost as xgb
from sklearn.metrics import f1_score

xgb_model = xgb.XGBClassifier()


xgb_model.fit(X_train, y_train)


y_xgb_model = xgb_model.predict(X_test)

f1_xgb_model = f1_score(y_pred=y_xgb_model, y_true=y_test_raw)

print("F1-score del modelo en el conjunto de prueba:", f1_xgb_model)


F1-score del modelo en el conjunto de prueba: 0.5095541401273885


In [ ]:
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer, f1_score
y_train = y_train_raw

param_dist = {
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 4, 5, 6, 7],
    'n_estimators': [150, 175, 200, 225, 250],
    'subsample': [0.8, 0.85, 0.9, 0.95, 1.0],
    'colsample_bytree': [0.7, 0.8, 0.9, 1.0],
    'gamma': [0.5, 0.6, 0.7, 0.8, 0.9],
    'min_child_weight': [1, 2, 3, 4, 5],
    'reg_alpha': [0.01, 0.1, 0.5, 1.0],
    'reg_lambda': [0.01, 0.1, 0.5, 1.0],
    'scale_pos_weight': [1, 2, 3, 4, 5]
}


xgb_clf = xgb.XGBClassifier(
    objective='binary:logistic',
    random_state=42
)

f1_scorer = make_scorer(f1_score)

random_search_xgb = RandomizedSearchCV(
    xgb_clf,
    param_distributions=param_dist,
    n_iter=10,
    scoring=f1_scorer,
    cv=5,
    verbose=1,
    n_jobs=-1
)


random_search_xgb.fit(X_train, y_train)


best_xgb_model = random_search_xgb.best_estimator_
best_params_xgb = random_search_xgb.best_params_


y_pred_best_xgb = best_xgb_model.predict(X_test)
f1_best_xgb = f1_score(y_pred=y_pred_best_xgb, y_true=y_test_raw)

print("Mejores hiperparámetros encontrados:", best_params_xgb)
print("F1-score del mejor modelo en el conjunto de prueba:", f1_best_xgb)


Fitting 5 folds for each of 10 candidates, totalling 50 fits
Mejores hiperparámetros encontrados: {'subsample': 0.85, 'scale_pos_weight': 2, 'reg_lambda': 0.5, 'reg_alpha': 0.5, 'n_estimators': 250, 'min_child_weight': 3, 'max_depth': 3, 'learning_rate': 0.05, 'gamma': 0.6, 'colsample_bytree': 0.7}
F1-score del mejor modelo en el conjunto de prueba: 0.5933232169954475


In [ ]:
import xgboost as xgb
from sklearn.metrics import f1_score
y_train = y_train_raw

params = {
    'objective': 'binary:logistic',
    'learning_rate': 0.01,
    'max_depth': 5,
    'n_estimators': 195,
    'subsample': 0.85,
    'colsample_bytree': 0.8,
    'gamma': 0.9,
    'min_child_weight': 1,
    'reg_alpha': 0.01,
    'reg_lambda': 1.0,
    'scale_pos_weight': 4,
    'random_state': 42
}

xgb_model = xgb.XGBClassifier(**params)

xgb_model.fit(X_train, y_train)

y_xgb_model = xgb_model.predict(X_test)

f1_xgb_model = f1_score(y_pred=y_xgb_model, y_true=y_test_raw)

print("Mejores hiperparámetros utilizados para cargar el modelo:", params)
print("F1-score del mejor modelo en el conjunto de prueba:", f1_xgb_model)

Mejores hiperparámetros utilizados para cargar el modelo: {'objective': 'binary:logistic', 'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 195, 'subsample': 0.85, 'colsample_bytree': 0.8, 'gamma': 0.9, 'min_child_weight': 1, 'reg_alpha': 0.01, 'reg_lambda': 1.0, 'scale_pos_weight': 4, 'random_state': 42}
F1-score del mejor modelo en el conjunto de prueba: 0.6026392961876833


## Preparacion del model para el test a entregar

In [ ]:
import xgboost as xgb
from sklearn.metrics import f1_score
y_train = y_train_raw

params = {
    'objective': 'binary:logistic',
    'learning_rate': 0.01,
    'max_depth': 5,
    'n_estimators': 195,
    'subsample': 0.85,
    'colsample_bytree': 0.8,
    'gamma': 0.9,
    'min_child_weight': 1,
    'reg_alpha': 0.01,
    'reg_lambda': 1.0,
    'scale_pos_weight': 4,
    'random_state': 42
}

model = xgb.XGBClassifier(**params)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

f1 = f1_score(y_pred=y_pred, y_true=y_test_raw)

print("F1-score del mejor modelo en el conjunto de prueba:", f1)

F1-score del mejor modelo en el conjunto de prueba: 0.6026392961876833


## Prediccion

In [ ]:
test = pd.read_parquet("/content/drive/MyDrive/TP2 - ORGANIZACIÓN DE DATOS/dataset_v2/test.parq", engine="pyarrow")


In [ ]:
test.head(10)

,attack_time,watcher_country,watcher_as_num,watcher_as_name,attacker_country,attacker_as_num,attacker_as_name,attack_type,watcher_uuid_enum,attacker_ip_enum
0,2023-07-31 07:17:58+00:00,US,54641.0,IMH-IAD,IN,14061.0,DIGITALOCEAN-ASN,http:spam,1,7696
1,2023-07-31 07:17:58+00:00,US,54641.0,IMH-IAD,IN,14061.0,DIGITALOCEAN-ASN,http:exploit,1,7696
2,2023-07-31 07:17:58+00:00,US,54641.0,IMH-IAD,IN,14061.0,DIGITALOCEAN-ASN,http:bruteforce,1,7696
3,2023-07-31 07:17:58+00:00,US,54641.0,IMH-IAD,IN,14061.0,DIGITALOCEAN-ASN,http:bruteforce,1,7696
4,2023-07-31 07:17:51+00:00,US,396982.0,GOOGLE-CLOUD-PLATFORM,US,53667.0,PONYNET,http:bruteforce,3,7543
5,2023-07-31 07:17:51+00:00,US,396982.0,GOOGLE-CLOUD-PLATFORM,US,53667.0,PONYNET,http:spam,3,7543
6,2023-07-31 07:17:51+00:00,US,396982.0,GOOGLE-CLOUD-PLATFORM,US,53667.0,PONYNET,http:exploit,3,7543
7,2023-07-31 07:17:57+00:00,DK,51468.0,One.com A/S,IN,14061.0,DIGITALOCEAN-ASN,http:exploit,7,7696
8,2023-07-31 07:17:57+00:00,DK,51468.0,One.com A/S,IN,14061.0,DIGITALOCEAN-ASN,http:spam,7,7696
9,2023-07-31 07:17:57+00:00,DK,51468.0,One.com A/S,IN,14061.0,DIGITALOCEAN-ASN,http:bruteforce,7,7696


# Preaparacion del dataset a predecir

In [ ]:
attack_types_df = (
    test.attack_type.str.split(":", expand=True)
    .rename(columns={0: "service", 1: "type"})
    .set_index(test["attacker_ip_enum"])
)

one_hot_attack_service_test = pd.get_dummies(

    attack_types_df.reset_index()
    .drop_duplicates(subset=["attacker_ip_enum", "service"])
    .set_index("attacker_ip_enum")["service"]
    # ,sparse=True
)
one_hot_attack_service_test = one_hot_attack_service_test.groupby("attacker_ip_enum").sum()

one_hot_attack_service_test = (one_hot_attack_service_test >= 1).astype(int)

one_hot_attack_service_test.rename(columns={'unknown': 'unknown_service'}, inplace=True)

one_hot_attack_service_test.head()

,database,ftp,http,pop3/imap,sip,smb,ssh,tcp,telnet,unknown_service,windows
attacker_ip_enum,,,,,,,,,,,
5,0,0,1,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,1,1,0,0,0
21,0,0,1,0,0,0,0,0,0,0,0
29,0,0,1,0,0,0,0,0,0,0,0
33,0,0,1,0,0,0,0,0,0,0,0


In [ ]:
one_hot_attack_types_test = pd.get_dummies(
    attack_types_df["type"],
    # sparse=True
)
one_hot_attack_types_test = one_hot_attack_types_test.groupby("attacker_ip_enum").sum()

one_hot_attack_types_test = one_hot_attack_types_test / one_hot_attack_types_test.sum(
    1
).values.reshape(-1, 1)

one_hot_attack_types_test.rename(columns={'unknown': 'unknown_type'}, inplace=True)

one_hot_attack_types_test.head(10)


,bruteforce,crawl,exploit,scan,spam,unknown_type
attacker_ip_enum,,,,,,
5,0.284848,0.0,0.321212,0.060606,0.333333,0.0
7,0.990000,0.0,0.000000,0.010000,0.000000,0.0
21,0.000000,0.0,0.000000,1.000000,0.000000,0.0
29,0.187500,0.0,0.437500,0.062500,0.312500,0.0
33,0.000000,0.0,0.000000,0.500000,0.500000,0.0
37,0.153846,0.0,0.564103,0.000000,0.282051,0.0
43,0.023810,0.0,0.261905,0.142857,0.571429,0.0
47,0.000000,0.0,0.500000,0.000000,0.500000,0.0
48,0.000000,0.0,0.043478,0.478261,0.478261,0.0


In [ ]:
test['day_of_week'] = pd.to_datetime(test['attack_time']).dt.day_name()
days_test = test[['attacker_ip_enum', 'day_of_week']].set_index('attacker_ip_enum')

one_hot_days_test = pd.get_dummies(

    days_test.reset_index()
    .set_index("attacker_ip_enum")["day_of_week"]
    # ,sparse=True
)

one_hot_days_test = one_hot_days_test.groupby("attacker_ip_enum").sum()

one_hot_days_test = one_hot_days_test / one_hot_days_test.sum(
    1
).values.reshape(-1, 1)
one_hot_days_test.head(10)


,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
attacker_ip_enum,,,,,,,
5,0.115152,0.200000,0.090909,0.127273,0.072727,0.193939,0.200000
7,0.005000,0.010000,0.000000,0.960000,0.015000,0.000000,0.010000
21,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
29,0.000000,0.125000,0.437500,0.000000,0.000000,0.250000,0.187500
33,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
37,0.307692,0.000000,0.000000,0.000000,0.692308,0.000000,0.000000
43,0.238095,0.000000,0.690476,0.071429,0.000000,0.000000,0.000000
47,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
48,0.000000,0.608696,0.000000,0.391304,0.000000,0.000000,0.000000


In [ ]:

test_df = test.drop_duplicates(["attacker_ip_enum"]).set_index("attacker_ip_enum").loc[:, ['watcher_as_num','attacker_as_num','attacker_country','watcher_country']]
#Conviero los '...as_num' a categorica para luego poder aplicarle un encoding acorde
test_df['watcher_as_num'] = test_df['watcher_as_num'].astype('category')
test_df['attacker_as_num'] = test_df['attacker_as_num'].astype('category')

In [ ]:
ports_features_df = ports_features_df.set_index('attacker_ip_enum')
protocols_features_df = protocols_features_df.set_index('attacker_ip_enum')

In [ ]:
X_test_df = pd.concat(
    [
        one_hot_attack_service_test,
        one_hot_attack_types_test,
        one_hot_days_test,
        ports_features_df,
        protocols_features_df,
        test_df,
    ],
    axis=1,
    join="inner",
)



In [ ]:
X_test_df.count()

database            49420
ftp                 49420
http                49420
pop3/imap           49420
sip                 49420
smb                 49420
ssh                 49420
tcp                 49420
telnet              49420
unknown_service     49420
windows             49420
bruteforce          49420
crawl               49420
exploit             49420
scan                49420
spam                49420
unknown_type        49420
Friday              49420
Monday              49420
Saturday            49420
Sunday              49420
Thursday            49420
Tuesday             49420
Wednesday           49420
port_1080           49420
port_110            49420
port_1194           49420
port_143            49420
port_1701           49420
port_1723           49420
port_20             49420
port_21             49420
port_22             49420
port_23             49420
port_25             49420
port_500            49420
port_53             49420
port_8080           49420
open_ports_c

In [ ]:
X_test_df.head()

,database,ftp,http,pop3/imap,sip,smb,ssh,tcp,telnet,unknown_service,...,port_53,port_8080,open_ports_count,protocol_tcp,protocol_udp,protocols_count,watcher_as_num,attacker_as_num,attacker_country,watcher_country
attacker_ip_enum,,,,,,,,,,,,,,,,,,,,,
5,0,0,1,0,0,0,0,0,0,0,...,0,0,2,1,0,1,15169.0,38235.0,KH,US
7,0,0,0,0,0,0,1,1,0,0,...,0,0,6,1,1,2,53667.0,16276.0,CA,LU
21,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,12897.0,3209.0,DE,DE
29,0,0,1,0,0,0,0,0,0,0,...,0,0,1,1,0,1,55293.0,132996.0,IN,US
33,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,14061.0,43037.0,CZ,GB


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

X_test_df['attacker_as_num'] = X_test_df['attacker_as_num'].map(means_attacker_as_num)
X_test_df['watcher_as_num'] = X_test_df['watcher_as_num'].map(means_watcher_as_num)

In [ ]:
from category_encoders.binary import BinaryEncoder
from sklearn.impute import SimpleImputer


# Imputar los valores nulos
X_test_df[columns_with_nans] = imputer.transform(X_test_df[columns_with_nans])

# Crear una nueva columna combinando "attacker_country" y "watcher_country"
X_test_df['combined_countries'] = X_test_df['attacker_country'] + '-' + X_test_df['watcher_country']

# Aplicar el BinaryEncoder a la nueva columna
X_test_df = encoder.transform(X_test_df)

# Eliminar las columnas originales "attacker_country" y "watcher_country"
X_test_df.drop(['attacker_country', 'watcher_country'], axis=1, inplace=True)


In [ ]:
X_test_df.head()

,database,ftp,http,pop3/imap,sip,smb,ssh,tcp,telnet,unknown_service,...,combined_countries_2,combined_countries_3,combined_countries_4,combined_countries_5,combined_countries_6,combined_countries_7,combined_countries_8,combined_countries_9,combined_countries_10,combined_countries_11
attacker_ip_enum,,,,,,,,,,,,,,,,,,,,,
5,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,1,0,1,1,0,0
7,0,0,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
21,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
29,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
33,0,0,1,0,0,0,0,0,0,0,...,1,0,1,0,1,0,1,1,0,0


In [ ]:
X_test = sc.transform(X_test_df)

In [ ]:
X_test_df.index.name

'attacker_ip_enum'

## Entrega de predicciones

In [ ]:
prediction_model2 = pd.Series(model.predict(X_test), index=X_test_df.index).rename("prediction_model2")

In [ ]:
prediction_model2.head()

attacker_ip_enum
5     0
7     0
21    0
29    0
33    0
Name: prediction_model2, dtype: int64

In [ ]:
prediction_model2.to_csv('/content/drive/MyDrive/TP2 - ORGANIZACIÓN DE DATOS/Predictions/prediction_model2.csv')

In [ ]:
len(prediction_model2)

49420

https://drive.google.com/drive/folders/1VHyzVuEJ1dy1rgDXhN9Vp3kqt3lGltiJ?usp=sharing